# 1. 프로젝트 주제

### 어플평점 4점을 넘기위해 고려해야 할 사항에 대한 ML Project

* 해결하고자하는 문제 : 어플 고평점 예측
* 데이터 : 캐글의 2017년 7월 앱스토어 어플 통계데이터(7197개)
* 선정이유
 * 코로나시국, 비대면 사회에서 가장 중요한 도구는 휴대폰이다.
 * 사용할 Target 특성은 4.5이상 평점을 받은 binary data로 분류문제이다.
 * 어떤 특징에 따라 고평점을 받을 수 있는지 없는지, 예측하는 모델링을 할 것이다.

# 2. 라이브러리 및 데이터 불러오기

In [ ]:
# 필요한 라이브러리들
import pandas as pd
import numpy as np
import graphviz
import matplotlib.pyplot as plt
import eli5

from category_encoders import OrdinalEncoder
from catboost import CatBoostClassifier

from eli5.sklearn import PermutationImportance

from imblearn.over_sampling import SMOTE, SMOTENC

from scipy.stats import boxcox, chi2_contingency

from sklearn.model_selection import train_test_split ,GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, roc_curve, roc_auc_score
from sklearn import tree, preprocessing
from sklearn.utils import resample

from pandas_profiling import ProfileReport

from pdpbox.pdp import pdp_interact, pdp_interact_plot, pdp_isolate, pdp_plot

In [ ]:
# https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps
# Data collection date (from API); July 2017

data = pd.read_csv('AppleStore.csv')
data.head()

In [ ]:
data.shape

### 특성별 설명(캐글)

"id" : 앱 ID  
"track_name": 앱 이름  
"size_bytes": 크기(바이트)  
"currency": 통화  
"price": 가격  
"ratingcounttot": 사용자 평가 수(모든 버전용)  
"ratingcountver": 사용자 평가 수(현재 버전용)  
"user_rating" : 평균 사용자 평가 값(모든 버전에 대해)  
"userratingver": 평균 사용자 평가 값(현재 버전의 경우)  
"ver" : 최신 버전 코드  
"cont_rating": 콘텐츠 등급  
"prime_genre": 기본 장르  
"sup_devices.num": 지원 장치 수  
"ipadSc_urls.num": 표현하기 위해 보여주는 스크린샷 수  
"lang.num": 지원되는 언어 수  
"vpp_lic": Vpp 장치기반 라이선스 사용  

# 3. 가설, 기준모델(Baseline Model), 평가지표설명

* 가설
    * 가설1 : 소프트웨어산업 중 가장 높은 비율을 차지하는 장르가 추천할 확률도 가장 높을 것이다
    * 가설2 : 가격의 영향력은 크기 때문에, 저렴하면 추천될 확률이 높을 것이다.
* 기준모델 및 평가지표설명
 * Target 특성은 어플 평점이 4점을 넘으면 True로 표시하는 새로운 특성을 생성.
 * Baseline Model로는 초기 최빈값인 0.9214에서 RandomForest의 AUC Score로 변경하였다.

# 4. EDA, Data Preprocessing

## 데이터를 아래와 같이 확인한 후, 함수로 전처리 및 EDA완료
###  1) 불필요한 특성제거 및 특성이름정리

* Currency특성에 다른 통화 유무 체크 => 전부 달러임을 확인  
data['currency'].unique()  

* 'Unnames: 0' 특성의 중복값여부체크 => 목차와 거의 동일하다 판단.(2개의 데이터가 다름)  
data.duplicated('vpp_lic').sort_values(ascending=True)  

* 컬럼의 단어 연결을 '.'을 '_'로 일관성있게 정리.


### 2) data 정리  

* 중복값여부체크 => 중복값이 데이터에 없음을 확인.  
data.duplicated().sort_values  

* 결측치유무확인->없음  
pd.isnull(data).sum()  

* ver 컬럼구성확인 -> 함수를 통해 int화함.  
data['ver'].value_counts()

* 어플이용 연령제한구성확인 -> 함수를 통해 int화함.  
data['cont_rating'].unique()  

* 장르별로 어플 구성 확인  
data['prime_genre'].value_counts()

* 대략적으로 데이터 구성내용 확인  
from pandas_profiling import ProfileReport  
profile = ProfileReport(data, minimal=True).to_notebook_iframe()  

* 불필요한 컬럼 일괄 삭제

In [ ]:
# ver 특성의 int화 위한 작업

# 알파벳을 리스트화하기

import string
string.ascii_lowercase   # 'abcdefghijklmnopqrstuvwxyz'

alphabet=list(string.ascii_letters)

# ver 함수생성 (str로 이뤄진 버전을 인트화)
def ver(value):
    if value[0]=='v':
        value = int(value[1])
        return value
    elif value[0]=='V':
        value = int(value[1])
        return value
    elif (value[0]=='9') and (value[1]=='9'):
        value = int(value[5])                 # 9999.2->버전2로간주
        return value
    else:                                     
        value = value[0]                      
        if value in alphabet:                 # 문자열이 있으면 0으로 변환
            value = int(0)
            return value
        else:
            value = int(value)                # 문자열이 없다면 int로 변환
            return value

# int 함수생성
def int_make(value):
    return int(value)

In [ ]:
# EDA AND Feature Engineering

def Feature_engineering(df):
    
    # 컬럼이름정리 ('.')->('_')
    df.columns = ['Unnamed: 0', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot',
                'rating_count_ver', 'user_rating_tot', 'user_rating_ver','ver', 'cont_rating', 'prime_genre', 
                'sup_devices_num', 'ipadSc_urls_num', 'lang_num', 'vpp_lic']

    # ver column int로 변경
    df['ver'] = df['ver'].apply(ver)
    
    # ver이 0이 아닌 것만 가져오기 =>value_counts()로 32개임을 확인했으나, 큰 의미가 없다 판단하여 제거함
    df = df[df.ver != 0]
    
    # cont_rating(연령) 안에 '+' 지우고, int화하기
    df['cont_rating'] = df['cont_rating'].str.split('+').str[0].apply(int_make)    
    
    # 'user_rating_tot'이 0이면 이상치로 봐서 해당 row 삭제 
    # (모든 버전에 대한 사용자평가인데, 어떤 회사라도 1개라도 있어야할텐데, 점수가 없다면 만들어진지 얼마 안된 어플일 것이라 간주하고 928개이지만 삭제함)
    df[df['user_rating_tot'] == 0.0]
    df = df[df.user_rating_tot > 0]
    
    # recommendation 타겟이 될 특성을 만들어 이진분류문제로 만듦.
    df['recommendation'] = (df['user_rating_tot']>4.5).astype(int) # True:1, False:0. 원하는 정수로 바꾸려면.replace({True:1,False:2})

    # 불필요한 5개 컬럼 삭제
    df = df.drop(['currency','Unnamed: 0', 'id','user_rating_ver', 'rating_count_ver'],axis=1)

    return df
data = Feature_engineering(data)
data.head()

In [ ]:
data.info()

In [ ]:
# High Cardinality 확인. track_name의 카디널리티가 높다. (unique가 높다)  
data.describe(exclude='number').T.sort_values(by='unique')  

In [ ]:
# 어플 이름이 같은 것이 있는지 확인
data[data['track_name'].duplicated(keep=False)==True]  

In [ ]:
data.shape

In [ ]:
# rating_count_tot이 적은것들 기준으로 삭제
data.drop(index=[7128], inplace=True)

In [ ]:
data.shape

## 3) 데이터분포확인

In [ ]:
# 데이터 분포확인
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(data['user_rating_tot'],kde=True);
plt.axvline(4, color='red'); 

In [ ]:
# 실제 평균점수보다 높은 어플은 몇개인가? True+False = 6268 => True와 False의 비율이 비슷해서 타겟 balance가 좋음.
print(data['recommendation'].value_counts())

print(data['recommendation'].value_counts(normalize=True))

In [ ]:
# recommendation의 imbalance 확인

%matplotlib inline
import seaborn as sns
sns.countplot(data['recommendation']);

* 최빈값인 0.9214를 초기 정확도 Baseline으로 잡아둔다.
* scale_pos_weight을 위해 ratio를 계산해둔다.
* Hyperparameter tuning만으로 부족하다면 smote도 고려해둔다.

In [ ]:
# scale_pos_weight을 위한 ratio
ratio0 = data['recommendation'].value_counts(normalize=True)[0]
ratio0

In [ ]:
ratio1 = data['recommendation'].value_counts(normalize=True)[1]
ratio1

In [ ]:
ratio = ratio1/ratio0
ratio

In [ ]:
data['user_rating_tot'].value_counts()

In [ ]:
data['rating_count_tot'].value_counts()

In [ ]:
# recommendation 특성 만들고 기반이 된 user_rating_tot, high cardinality인 track_name 제거
data.drop(['user_rating_tot','track_name'], axis=1, inplace=True)

# data reset_index
data = data.reset_index(drop=True)
data.head()

In [ ]:
# 특성간 상관관계 시각화
new = data['recommendation']
trainn = data.drop(['recommendation'],axis=1)
trainn = pd.concat([new,trainn], axis=1)
plt.figure(figsize=(10,10))
sns.heatmap(data=trainn.corr(), annot=True, fmt='.2f', linewidths=.5, cmap='Blues');

* 특성들간 상관관계가 높음이 확인되지 않음.

In [ ]:
# recommendation True로 나온 특성별 분포확인
recommendd = trainn[trainn.recommendation == True]
print(recommendd.shape)


In [ ]:
# recommendation True로 나온 특성별 분포확인

for i in trainn.columns:
    sns.displot(recommendd[i],kde=True) # kde : 커널밀도추정. 커널함수와 데이터를 바탕으로 연속성있는 확률밀도함수를 추정하는것.

### 가설1 소프트웨어산업 중 가장 높은 비율을 차지하는 장르가 추천할 확률도 가장 높을 것이다

In [ ]:
# 장르별 2021년 7월 어플비중비율

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 15

pie, ax = plt.subplots(figsize=[10,10])
plt.title("2021년 7월 어플 장르별 비율", fontsize=30);

# define data
data_by_genre=data['prime_genre'].value_counts()
labels=data['prime_genre'].value_counts().index

colors = sns.color_palette('pastel')

#create pie chart
plt.pie(data_by_genre, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

In [ ]:
# 어플이용자 중 많은 비율이 게임이므로, 게임의 추천비율이 많을 것이다.

good = data[data.recommendation==1]

# 장르별 2021년 7월 어플비중비율

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 15

pie, ax = plt.subplots(figsize=[10,10])
plt.title("추천된 어플 장르별 비율", fontsize=30);

# define data
data_by_genre=good['prime_genre'].value_counts()
labels=data['prime_genre'].value_counts().index

colors = sns.color_palette('pastel')

#create pie chart
plt.pie(data_by_genre, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

In [ ]:
# 어플이용자 중 많은 비율이 게임이므로, 게임의 추천비율이 많을 것이다.

bad = data[data.recommendation==0]

# 장르별 2021년 7월 어플비중비율

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 15

pie, ax = plt.subplots(figsize=[10,10])
plt.title("추천되지 않은 어플 장르별 비율", fontsize=30);

# define data
data_by_genre=bad['prime_genre'].value_counts()
labels=data['prime_genre'].value_counts().index

colors = sns.color_palette('pastel')

#create pie chart
plt.pie(data_by_genre, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

In [ ]:
# 실제 총 데이터 중 추천비율을 보자 => 게임의 추천이 가장 높긴 하지만, 비추천은 훨씬 높음을 확인.

obs = pd.crosstab(data['prime_genre'],data['recommendation'])
obs

import matplotlib.pyplot as plt
%matplotlib inline

ax = obs.plot(kind='bar', title='recommendation by genre1', figsize=(8, 6))
ax.set_ylabel('count')
plt.grid(color='darkgray')
plt.show()

In [ ]:
# 장르별 추천비율

obs['ratio'] = obs[1]/(obs[1]+obs[0])
obss = obs.drop([0,1],axis=1)

import matplotlib.pyplot as plt
%matplotlib inline

ax = obss.plot(kind='bar', title='recommendation ratio by genre2', figsize=(8, 6))
ax.set_ylabel('count')
plt.grid(color='darkgray')
plt.show()


* 어플 장르에 게임이 차지하는 비율이 압도적으로 높다.
* 전체를 기준으로 추천율, 비추천율을 장르별로 구분하여도 게임이 다른 장르에 비해서 모두 높다.
* 마지막 그래프인 recommendation ratio by genre2를 보면, 게임의 점유율과 무관하게 추천율은 높지 않음을 확인할 수 있으며, 가장 추천율이 높은 장르는 Book으로 확인하였다.

### 가설 2 : 가격의 영향력은 크기 때문에, 저렴하면 추천될 확률이 높을 것이다.

In [ ]:
# 추천된것 중 가격비교함.

good = data[data.recommendation==1]
good.shape

good_by_price = good['price'].value_counts().reset_index().rename(columns={'index':'price','price':'count'})

plt.figure(figsize=(10,10))
sns.barplot(x='price',y='count',data=good_by_price)
plt.title('가격기준 추천 수',fontsize=20);

In [ ]:
# 실제 총 데이터 중 추천비율을 보자 => 무료의 비중이 가장 높긴 하지만, 비추천은 훨씬 높음을 확인.

obs2 = pd.crosstab(data['price'],data['recommendation'])
obs2

import matplotlib.pyplot as plt
%matplotlib inline

ax = obs2.plot(kind='bar', title='recommendation by price', figsize=(8, 6))
ax.set_ylabel('count')
plt.grid(color='darkgray')
plt.show()


In [ ]:
# 가격별 추천비율

obs2['ratio'] = obs2[1]/(obs2[1]+obs2[0])
obs3 = obs2.drop([0,1],axis=1)

import matplotlib.pyplot as plt
%matplotlib inline

ax = obs3.plot(kind='bar', title='recommendation ratio by price', figsize=(8, 6))
ax.set_ylabel('count')
plt.grid(color='darkgray')
plt.show()


In [ ]:
data[data['price']==18.99]

In [ ]:
data[data['price']==23.99]

* 추천과 비추천의 데이터 수가 무료에 다른 가격에 비해 많고, 추천과 비추천 모두 가격이 적을수록 수치가 높았다.
* 큰 의미가 없다 판단하여 각 가격별로 추천비율을 구했을 때, 18.99달러와 23.99달러의 추천율이 1로, 데이터가 하나씩있는 이상치임을 확인하였다.
* 해당 가설은 큰 의미가 없는 것으로 판단한다.

# 5. Modeling

### 모델의 유용성과 한계

* 목적 : 만들고자 하는 모델의 경우 어떤 특성을 가진 어플이 4.5이상의 추천을 받을 수 있는지 예측
* 유용성 : 어플을 사용하려는 고객은 추천여부를 알 수 있고, 앱스토어 자체에서 추천하는 항목으로 될 수 있기에 판매자에게도 유용할 것이다.
* 한계 : 
 1. 악의적으로 저점을 받는 어플은 추천할 수 있는 가능성을 상실할 수 있다.
 2. paid/Free의 구분이 어플구입시 기준으로 나뉘는데, 시범기간 이후 유료화로 변하기도하고, 무료버전이라하더라도 유료를 해야 가능한 서비스가 있다면 구분이 모호하다 할 수 있다.
 3. 데이터가 2021/7월 한 달치로 부족할 수 있으나, 특별한 어플들이 많이 나오지 않은 이상, 특징은 크게 다르지 않을 것이라 판단한다.

## 1) 1차 모델링(RandomForest), 기준모델

In [ ]:
# 훈련/검증/테스트셋 나누기

train, test = train_test_split(data, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)

target = 'recommendation'
features = data.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

train.shape, val.shape, test.shape

In [ ]:
# randomforest_ordinal encoding

pipe_rf_o = make_pipeline(
    OrdinalEncoder(), # 범주형 자료를 모델링할 때는 원핫보다 오디널이 좋음! 중요한 노드가 상위에서 선택되어야하는데 원핫을하면 적용이 잘 안됨! 한가지 특성이 여러가지로 나눠지기 때문에! 그래서 노미널인코딩이라도 오디널을 사용함! 트리에서는 순서가 상관없어서 괜찮다!
    RandomForestClassifier())

pipe_rf_o.fit(X_train, y_train)
y_pred = pipe_rf_o.predict(X_val)

print('훈련 정확도', pipe_rf_o.score(X_train,y_train))
print('검증 정확도', pipe_rf_o.score(X_val, y_val))
print('F1 score', f1_score(y_val, y_pred))
print('AUC 점수 :', roc_auc_score(y_val, y_pred))
print('REPORT',classification_report(y_val, y_pred))

* Ordinal Encoder로 RandomForest를 이용해서 1차 Baseline Modeling을 진행했다.
* 기본적인 모델링으로 특별한 Parameter없이 진행했으며, 훈련정확도가 1로 과적합임을 확인하였다.
* 하지만 검증 Set에서의 정확도가 최빈값 Baseline을 넘었으므로 이번 Modeling에서 유심히 볼 평가지표인 'AUC Score'를 Baseline으로 두고 진행한다.
* Target의 imbalance함이 score에 영향을 주어 소수인 '1'에 대한 score는 전체적으로 낮은데, 밸런스화가 이번 모델링의 핵심으로 보인다.
* 추후 SMOTE를 적용하면 얼마나 성능이 좋아질 지 확인해보기로 한다.

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(pipe_rf_o, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15)
plt.show()

In [ ]:
tp = 5
tn = 919
fp = 5
fn = 69
total = tp+tn+fp+fn

In [ ]:
y_train.value_counts(normalize=True)[1]

## 2) 2차 모델링 (CatBoost)

In [ ]:
# catboost, ordinalencoder

pipe_cat_o = make_pipeline(
    OrdinalEncoder(), # 범주형 자료를 모델링할 때는 원핫보다 오디널이 좋음! 중요한 노드가 상위에서 선택되어야하는데 원핫을하면 적용이 잘 안됨! 한가지 특성이 여러가지로 나눠지기 때문에! 그래서 노미널인코딩이라도 오디널을 사용함! 트리에서는 순서가 상관없어서 괜찮다!
    CatBoostClassifier())

pipe_cat_o.fit(X_train, y_train)
y_pred = pipe_cat_o.predict(X_val)

print('훈련 정확도', pipe_cat_o.score(X_train,y_train))
print('검증 정확도', pipe_cat_o.score(X_val, y_val))
print('F1 score', f1_score(y_val, y_pred))
print('AUC 점수 :', roc_auc_score(y_val, y_pred))
print('REPORT',classification_report(y_val, y_pred))

## 3) Hyper Parameter Tuning the RandomForest with SMOTE

In [ ]:
###### pipe라인 랜덤포레스트 모델의 파라미터튜닝
pipe = make_pipeline(
      OrdinalEncoder(),
      RandomForestClassifier(random_state=2))

# 튜닝할 하이퍼파라미터의 범위를 지정
parameters = {'randomforestclassifier__max_depth': range(1, 5, 2), 
              'randomforestclassifier__max_features': range(1, 5, 2), 
              'randomforestclassifier__min_samples_leaf' : range(1, 5, 2)}
    
# 최적의 hyper parameter를 찾기 위한 RandomizedSearchCV
rf_classifier = RandomizedSearchCV(pipe, 
                                    param_distributions=parameters, #  param_distributions : 사전/사전목록
                                    n_iter=10, # 정수, 기본값=10
                                    cv=5, # 교차검증생성기 또는 반복가능 default=None
                                    scoring='accuracy', # 평가방법
                                    verbose=1) # 진행상황표시. 높을수록 더 많은 메세지표시.
rf_classifier.fit(X_train, y_train);

In [ ]:
# RandomSearchCV 결과 확인
print('Best Parameters: ', rf_classifier.best_params_)
print('검증정확도: ', rf_classifier.best_score_)

### (1) RandomForest with SMOTE1

In [ ]:
# sampling_strategy=1.0

# 인코딩
encoder = OrdinalEncoder()
dataaa=encoder.fit_transform(data)

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2, sampling_strategy=1.0) # sampling_strategy = 'minority'
X_train, y_train = sm.fit_resample(X_train, y_train)

# Hyperparameter 적용
smote = RandomForestClassifier(random_state=2, min_samples_leaf=1, max_features=1, max_depth=1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### (2) RandomForest with SMOTE2

In [ ]:
# sampling_strategy=1.0, class_weight

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2, sampling_strategy=1.0) # sampling_strategy = 'minority'
X_train, y_train = sm.fit_resample(X_train, y_train)

# classweight, Hyperparameter 적용
smote = RandomForestClassifier(class_weight={0:ratio0,1:ratio1}, random_state=2, min_samples_leaf=1, max_features=1, max_depth=1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### (3) RandomForest with SMOTE3

In [ ]:
# class_weight

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2)
X_train, y_train = sm.fit_resample(X_train, y_train)

# classweight, Hyperparameter 적용
smote = RandomForestClassifier(class_weight={0:ratio0,1:ratio1}, random_state=2, min_samples_leaf=1, max_features=1, max_depth=1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### RandomForest의 Smote 및 Hyperparameter tuning 결과
* RandomizedSearchCV를 이용하여 hyper parameter를 적용하고, imbalance를 살펴보기로 한다.
* RandomForest의 경우 Hyperparameter에 class_weight={0:ratio0,1:ratio1}를 사용시, AUC score가 0.5이다.
* 소수인 ‘1’에 해당하는 metric socre가 반영이 되지 않았기 때문이다. 
* RandomForest의 class_weight을 제외하고, smote의 parameter인 sampling_strategy=1 만 적용한 첫번째 모델이 AUC가 0.57로 가장 훌륭하다.

## 4) Hyper Parameter Tuning the CatBoost with SMOTE

In [ ]:
train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)

target = 'recommendation'
features = data.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

model = CatBoostClassifier(random_state=2)

grid = {'learning_rate': [0.03, 0.1],
        'max_depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

randomized_search_result = model.randomized_search(grid, X=X_train, y=y_train, plot=True)

In [ ]:
print(randomized_search_result)

### (1) CatBoost with SMOTE1

In [ ]:
# 합성샘플생성1 (smote1) CatBoost sampling


# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2, sampling_strategy=1.0) # sampling_strategy = 'minority'
X_train, y_train = sm.fit_resample(X_train, y_train)

smote = CatBoostClassifier(random_state=2,depth=6, l2_leaf_reg=5, learning_rate=0.1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### (2) CatBoost with SMOTE2

In [ ]:
# sampling_strategy, Class_weights

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2, sampling_strategy=1.0) # sampling_strategy = 'minority'
X_train, y_train = sm.fit_resample(X_train, y_train)

smote = CatBoostClassifier(class_weights={0:ratio0,1:ratio1}, random_state=2,depth=6, l2_leaf_reg=5, learning_rate=0.1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### (3) CatBoost with SMOTE3

In [ ]:
# class_weights


# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)
 
target = 'recommendation'
features = dataaa.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# smote 적용
sm = SMOTE(random_state=2)
X_train, y_train = sm.fit_resample(X_train, y_train)

smote = CatBoostClassifier(class_weights={0:ratio0,1:ratio1},random_state=2,depth=6, l2_leaf_reg=5, learning_rate=0.1).fit(X_train,y_train)
smote_pred=smote.predict(X_val)

print('훈련 정확도', smote.score(X_train, y_train))
print('검증 정확도', smote.score(X_val, y_val))
print('f1 스코어',f1_score(y_val, smote_pred))
print('auc점수 : ', roc_auc_score(y_val, smote_pred))
print('Report \n',classification_report(y_val, smote_pred))

### CatBoost의 Smote 및 Hyperparameter tuning 결과

* RanomizedSearch를 이용해서 Hyper parameter를 적용하고 imbalance를 살펴보기로 한다.
* CatBoost의 class_weight없이 SMOTE의 sampling_strategy=1만 사용한 첫번째 모델이 AUC가 0.5397로 가장 좋다.
* CatBoost의 경우 Hyperparameter에 class_weight={0:ratio0,1:ratio1}를 사용시, SMOTE의 sampling_strategy hyperparameter에 영향을 미치지 않았다.
* 이에 따라 두번째 모델과 세 번째 모델이 모든 metric score에서 동일함을 보이며, AUC가 0.53으로 첫번째 모델보다 성능이 낮음을 확인할 수 있다

## 5) Hyper Parameter Tuning the RandomForest with OverSampling

In [ ]:
# high cardinality가진 column에 유용한 Ordinal Encoding process

encoder = OrdinalEncoder()
dataaa = encoder.fit_transform(data)

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)

target = 'recommendation'
features = data.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# concatenate our traing data back together
X = pd.concat([X_train, y_train],axis=1)

# seperate minority and majority classes
no_recommendd = X[X.recommendation==0]
yes_recommendd = X[X.recommendation==1]

# upsample minority
yes_reco_upsampled = resample(yes_recommendd, 
                              replace=True, # sample with replacement
                             n_samples = len(no_recommendd),# match number in majority class
                             random_state=2) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([no_recommendd, yes_reco_upsampled])

# check new recommendation counts
upsampled.recommendation.value_counts() # 동일하게 맞음.


In [ ]:
# class_weight미적용.

y_train = upsampled.recommendation
X_train = upsampled.drop(target, axis=1)

# randomforest에 class_weight={0:ratio0, 1:ratio1} 적용시, 훈련정확도 및 AUC 0.5로 나옴.
upsampled_model = RandomForestClassifier(random_state=2,min_samples_leaf=1, max_features=1, max_depth=1)
upsampled_model.fit(X_train,y_train)

upsampled_model_pred = upsampled_model.predict(X_val)

# Checking accuracy

print('훈련 정확도', upsampled_model.score(X_train, y_train))
print('검증 정확도', upsampled_model.score(X_val, y_val))
print('F1 스코어',f1_score(y_val, upsampled_model_pred))
print('auc점수 : ', roc_auc_score(y_val, upsampled_model_pred))
print('Report \n',classification_report(y_val, upsampled_model_pred))

### RandomForest의 모델 중 가장 우수한 모델 : OverSampling 적용된 모델(AUC:0.59)

* Hyper Parameter가 동일하게 적용된 모든 RandomForest Model을 확인시, OverSampling이 적용된 모델이 훈련정확도가 0.62로 가장 낮고, AUC가 0.59로 가장 높다.
* SMOTE로 적용된 모델 중 가장 우수했던 첫번째 모델과 비교하면, 두 모델을 비교하면 SMOTE적용모델이 훈련정확도 0.7로 0.08 더 정확하지만, baseline을 AUC로 뒀기에 OverSampling을 적용한 모델을 RandomForest를 가장 우수하다고 할 수 있다.

## 6) Hyper Parameter Tuning the CatBoost with OverSampling

In [ ]:
# high cardinality가진 column에 유용한 Ordinal Encoding process

encoder = OrdinalEncoder()
dataaa = encoder.fit_transform(data)

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)

target = 'recommendation'
features = data.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# concatenate our traing data back together
X = pd.concat([X_train, y_train],axis=1)

# seperate minority and majority classes
no_recommendd = X[X.recommendation==0]
yes_recommendd = X[X.recommendation==1]

# upsample minority
yes_reco_upsampled = resample(yes_recommendd, 
                              replace=True, # sample with replacement
                             n_samples = len(no_recommendd),# match number in majority class
                             random_state=2) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([no_recommendd, yes_reco_upsampled])

# check new recommendation counts
upsampled.recommendation.value_counts() # 동일하게 맞음.


### (1) CatBoost with OverSampling1

In [ ]:
# Catboost에 class_weight 적용

y_train = upsampled.recommendation
X_train = upsampled.drop(target, axis=1)

upsampled_model = CatBoostClassifier(class_weights={0:ratio0, 1:ratio1},random_state=2,depth=6, l2_leaf_reg=5, learning_rate=0.1)
upsampled_model.fit(X_train,y_train)

upsampled_model_pred = upsampled_model.predict(X_val)

# Checking accuracy

print('훈련 정확도', upsampled_model.score(X_train, y_train))
print('검증 정확도', upsampled_model.score(X_val, y_val))
print('F1 스코어',f1_score(y_val, upsampled_model_pred))
print('auc점수 : ', roc_auc_score(y_val, upsampled_model_pred))
print('Report \n',classification_report(y_val, upsampled_model_pred))

### (2) CatBoost with OverSampling2

In [ ]:
# Catboost에 class_weight 미적용.

y_train = upsampled.recommendation
X_train = upsampled.drop(target, axis=1)

upsampled_model = CatBoostClassifier(random_state=2,depth=6, l2_leaf_reg=5, learning_rate=0.1)
upsampled_model.fit(X_train,y_train)

upsampled_model_pred = upsampled_model.predict(X_val)

# Checking accuracy

print('훈련 정확도', upsampled_model.score(X_train, y_train))
print('검증 정확도', upsampled_model.score(X_val, y_val))
print('F1 스코어',f1_score(y_val, upsampled_model_pred))
print('auc점수 : ', roc_auc_score(y_val, upsampled_model_pred))
print('Report \n',classification_report(y_val, upsampled_model_pred))

### CatBoost의 모델 중 가장 우수한 모델 : OverSampling 적용된 모델(AUC:0.54)

* OverSampling의 CatBoost는 class_weights={0:ratio0, 1:ratio1} 을 적용하지 않은 모델이 AUC 0.54로 우수하다.

# 6. 머신러닝모델 해석결과

## 1) 최종모델 : OverSampling 적용된 RandomForest모델

In [ ]:
# high cardinality가진 column에 유용한 Ordinal Encoding process

encoder = OrdinalEncoder()
dataaa = encoder.fit_transform(data)

# 훈련/검증/테스트셋 나누기

train, test = train_test_split(dataaa, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=2)

target = 'recommendation'
features = data.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

# concatenate our traing data back together
X = pd.concat([X_train, y_train],axis=1)

# seperate minority and majority classes
no_recommendd = X[X.recommendation==0]
yes_recommendd = X[X.recommendation==1]

# upsample minority
yes_reco_upsampled = resample(yes_recommendd, 
                              replace=True, # sample with replacement
                             n_samples = len(no_recommendd),# match number in majority class
                             random_state=2) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([no_recommendd, yes_reco_upsampled])

# check new recommendation counts
upsampled.recommendation.value_counts() # 동일하게 맞음.


In [ ]:
# class_weight미적용.

y_train = upsampled.recommendation
X_train = upsampled.drop(target, axis=1)

# randomforest에 class_weight={0:ratio0, 1:ratio1} 적용시, 훈련정확도 및 AUC 0.5로 나옴.
upsampled_model = RandomForestClassifier(random_state=2,min_samples_leaf=1, max_features=1, max_depth=1)
upsampled_model.fit(X_train,y_train)

upsampled_model_pred = upsampled_model.predict(X_val)

# Checking accuracy

print('훈련 정확도', upsampled_model.score(X_train, y_train))
print('검증 정확도', upsampled_model.score(X_val, y_val))
print('F1 스코어',f1_score(y_val, upsampled_model_pred))
print('auc점수 : ', roc_auc_score(y_val, upsampled_model_pred))
print('Report \n',classification_report(y_val, upsampled_model_pred))

In [ ]:
# roc_curve(타겟값, prob of 1)

model = upsampled_model

y_pred_proba = model.predict_proba(X_val)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})
# print(roc)

# roc 시각화
plt.rcParams["figure.figsize"] = (10,4)
plt.subplot(121)
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print('idx:', optimal_idx, ', threshold:', optimal_threshold)

# auc 시각화
plt.subplot(122)
plt.plot(tpr-fpr);

# threshold 설정 및 레포트
y_pred_optimal = y_pred_proba >= optimal_threshold
print('Report \n',classification_report(y_val, y_pred_optimal))

# auc 점수
auc_score = roc_auc_score(y_val, y_pred_optimal)
print('최종 검증 정확도: ', accuracy_score(y_val, y_pred_optimal))
print('최종 f1 스코어',f1_score(y_val, y_pred_optimal))
print('최종 auc점수 : ', auc_score)

In [ ]:
# 테스트 데이터 성능확인
# 파이프라인 빼고 재구성해서 나타냄

model = upsampled_model

# roc_curve(타겟값, prob of 1)
y_pred_proba = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})

# roc 시각화
plt.rcParams["figure.figsize"] = (10,4)
plt.subplot(121)
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

# 최적의 threshold
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print('idx:', optimal_idx, ', threshold:', optimal_threshold)

# auc 시각화
plt.subplot(122)
plt.plot(tpr-fpr);

# threshold 설정 및 레포트
y_pred_optimal = y_pred_proba >= optimal_threshold
print('Report \n',classification_report(y_test, y_pred_optimal))

# auc 점수
auc_score = roc_auc_score(y_test, y_pred_optimal)
print('테스트 정확도', model.score(X_test, y_test))
print('f1 스코어',f1_score(y_test, y_pred_optimal))
print('auc점수 : ', auc_score)

## 2) Feature Importance

* 트리기반 앙상블모델에서 사용되는 중요도로, 모든 상호학습 후에 특성들의 중요도 정보(Gini importance)를 이용한다.
* 노드들의 지니불순도(Gini Impurity)를 가지고 계산하는데, 노드가 중요할수록 불순도가 크게 감소하는 사실을 이용한다.
* 속도는 빠르지만 high-cardinality특성이 있는 경우, 트리구성 중 분기에 이용될 확률이 높아 과적합의 위험이 있다.

In [ ]:
dataaaa=dataaa.drop(['recommendation'],axis=1)

In [ ]:
# 특성중요도확인

importances = pd.Series(upsampled_model.feature_importances_, dataaaa.columns)
plt.figure(figsize=(10,10))
importances.sort_values().plot.barh();

## Permutation Importance (순열 중요도)

* 모델 예측에 가장 큰 영향을 미치는 Feature를 파악하기 위해, 모델을 학습시킨 후, 훈련모델이 특정 Feature를 안썼을 때, 성능 손실이 얼마나 영향을 주는지, 변수중요도를 확인하는 방법. 
* high-cardinality의 경우 생기는 문제를 보완하기 위해 관심있는 특성에만 무작위로 노이즈(shuffle, permutation)를 주고, 평가지표가 얼마나 감소하는지 측정.(Black-box모델에 대해서 특정 feature를 안썼을 때, 성능이 얼마만큼 영향을 미치는지 feature의 중요도를 확인.)  


* 장점
* 모델을 재학습시킬 필요가 없다. (feature를 제거하는 대신, 무작위로 섞어(permutation) 노이즈로 만들기 때문에)
* 이렇게 특정 Feature를 섞었을 때, 예측값이 실제값보다 얼마나 차이나는지를 통해 해당 Feature의 영향력을 파악할 수 있다.
* 훈련된 모델이 이 특정 Feature에 크게 의존하고 있을 경우, 예측 정확도는 감소한다.(모델성능이 떨어지면 중요한 Feature)  


* 단점
* feature 값들을 무작위로 섞어 feature간의 연결되는 중요도가 사라지기에, 실행마다 Feature importance 결과가 매우 달라질 수 있고, 비현실적인 데이터 조합이 생성될 가능성이 높다.

In [ ]:
# 순열중요도로 중요하지 않은 순서대로 나타냄
# 파이프라인 빼고 재구성해서 나타냄

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# permuter 정의
permuter = PermutationImportance(
    upsampled_model, # model
    scoring='roc_auc', # metric. 
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state=2)

'''# permuter 계산은 preprocessing 된 X_val을 사용
X_val_transformed = pipe_cat_o.named_steps['preprocessing'].transform(X_val)'''

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업
permuter.fit(X_val, y_val)

feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()


In [ ]:
# 특성별 score 확인->ver이 모델에서 가장 높은 특성임을 보여줌
# 3차모델링의 파이프라인으로 나타냄

eli5.show_weights(permuter, top=None,
    feature_names=feature_names 
)

* 순열 중요도에 대한 음수 값은 데이터에 대한 예측이 실제 데이터보다 더 정확함을 나타낸다. 
* 이는 특성이 예측에 많은 기여를 하지 않고,(중요도가 0에 가까움) 무작위 우연으로 인해 섞인 데이터에 대한 예측이 더 정확하다는 것을 의미한다.

Feature Importance는 트리기반 앙상블모델에서 사용되는 중요도로 노드가 중요할수록 불순도가 크게 감소하는 특징을 이용하여 중요도를 나타낸 것으로,  
lang_num, prime_genre, size_bytes Feature가 중요함을 보여준다.  
Permutation Importance는 무작위로 섞어 노이즈로 만들기에 예측값과 실제값이 어람나 차이나는지 영향력을 파악하는 것인데,  
rating_count_tot, size_bytes, ipadSc_urls_num의 순으로 중요한 Feature임을 보여준다.

## 4) PDP

* Feature Importance에서 중요한 상위 4가지 Feature에 대해 PDPlot 확인

In [ ]:
# dpi(dots per inch) 수치조정-> 이미지 화질조정
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 144

for i in ['size_bytes', 'rating_count_tot', 'size_bytes','ver']:
    feature = i
    isolated = pdp_isolate(
        model = upsampled_model,
        dataset=X_train,
        model_features=X_train.columns,
        feature=feature,
        grid_type='percentile',
        num_grid_points=10
        )
    pdp_plot(isolated,feature_name=feature);

In [ ]:
# price 0~10로 확대 
pdp_plot(isolated, feature_name='price')
plt.xlim((0,10));

## 5) 프로젝트 회고

* Feature Importance와 Permutation Importance에서의 특성이 비슷하다고 하기는 어려웠고, PDP에서는 추천이 안 되는 시각화는 나왔지만, 추천되는 부분에 대한 시각화가 잘 나타나지 않았다. 
* Imbalance한 데이터의 모델링에 성능을 높이기 위해 많은 시도를 하였으나, 최종모델에 Test Data를 넣었을 때, AUC Score가 0.58로 마무리 지은 것이 아쉽다.
* 모델의 성능이 좋다고 하기는 어려운데, 이는 데이터 특성의 정보가 부족하기 때문이라고 생각한다.
* 어플의 추천시 특성상 주관적인 판단이 많이 들어가는데, 캐글 데이터로 주관적인 내용을 구분하기는 쉽지 않았다.
* 연령대나 성별 등 이용자의 정보가 추가적으로 특성에 반영된다면 더 좋은 모델링을 할 수 있을 것이라 기대한다.